# APS PYTHON PARA CIÊNCIA DE DADOS

Integrantes:
- Daniel Costa Delattre 
- Dener Vilarinho
- Renan Buck
- Gabriel Peres Afonso


## Objetivo 
O propósito desse trabalho é modelar preço de uma casa conforme suas características disponíveis no Dataset utilizado. Segundo a publicação de João Pedro Pinheiro Malere, Publio Netto de Almeida e Humberto Hayashi Sano, __a utilização de modelos em aprendizagem de máquina para o modelo imobiliário tem sido realizada de maneira extensiva__, isso é possível dado o avanço da área da computação. A finalidade de se ter um modelo preditivo para preços de casa é obter uma forma de automatizar o processo de avaliação do preço de uma casa quando um possível cliente quer colocar seu imóvel a venda, assim, poupando tempo de decisão e de análise, mas executando-a de maneira confiável. 

Carregando algumas bibliotecas:

In [36]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os, csv, string, re
import statsmodels.api as sm
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor


In [37]:
### Função utilitária para fazer a regressão linear com constante adicionada
def regress(X,Y):
    '''
    X: coluna(s) do DataFrame que serão utilizadas como variável(is) preditoras/explicativas (independente(s))
    Y: coluna do DataFrame que será utilizada como variável resposta (dependente)
    '''
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

#função para calcular a acurácia
def evaluate(model, test_features, test_labels):
    #faz a predição do valor das casa utilizando o modelo
    predictions = model.predict(test_features)
    #tira o erro absoluto entre valor previsto e o real
    errors = abs(predictions - test_labels)
    #Porcentagem média do erro absoluto
    mape = 100 * np.mean(errors / test_labels)
    #Acurácia do modelo
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: ${:0.2f} .'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    # rf_random.fit(xtreino, ytreino)
    return accuracy

def load_data():
    ROOT = 'data'
    fname = 'kc_house_data.csv'
    fpath = f'{ROOT}/{fname}'
    df = pd.read_csv(fpath).drop(['date','zipcode'], axis=1)
    return df

#### Carregando a base de dados de teste e de treino e excluindo as colunas date e zipcode (pois não serão usadas nas análise):

In [44]:
df = load_data()

In [39]:
#Printando as colunas que temos na variável do dataset
df.columns.to_list()

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

Mas afinal, o que cada uma delas significa?
- **ID:**          Numero identificação da casa 
- **Price:**       Preço
- **Bedrooms:**    Número de quartos
- **Bathrooms:**   Números de banheiros
- **Sqft_living:** Área habitável
- **Sqft_lot:**    Tamanho do terreno
- **Floors:**      Andares
- **Waterfront:**  Se tem vista para o mar
- **View:** Nota para a vista 0-4
- **Condition:** Condição 0-5
- **Grade:** Classificação 
- **Sqft_above:** Metros quadrados que descontados o porão
- **Sqft_basement:** Tamanho do porão 
- **Yr_built:** Ano de construção
- **Yr_renovated:** Ano de reconstrução/reforma
- **Lat:** Latitude
- **Long:** Longitude
- **Sqrt_living15:** Metros quadrados habitáveis no ano de 2015
- **Sqrt_lot15:** Metros quadrados do terreno todo no ano de 2015

Verificando se não existe linhas sem dados/informação

In [40]:
np.sum(df.isnull())

id               0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## Visualizando o dataset

In [41]:
df.head(10)

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,5631500400,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503
5,7237550310,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,47.6561,-122.005,4760,101930
6,1321400060,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,47.3097,-122.327,2238,6819
7,2008000270,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,47.4095,-122.315,1650,9711
8,2414600126,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,47.5123,-122.337,1780,8113
9,3793500160,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,47.3684,-122.031,2390,7570


#### Removendo Outliers


In [45]:
def remove_outliers(df_):
    df = df_.copy()
    #Criando o Isolation Forest
    clf = IsolationForest(max_samples = 110, random_state = 42)
    # Isolation forest retorna o score de anomalia de cada sampl

    #treinando o modelo
    clf.fit(df)
    y_semout = clf.predict(df)
    y_semout = pd.DataFrame(y_semout, columns = ['Top'])
    #processo para remover as linhas com outliers do banco de dados
    mask_remove_outliers = y_semout[y_semout['Top'] == 1].index.values
    df = df.iloc[mask_remove_outliers]
    df.reset_index(drop=True, inplace=True)
    
    print("Número de outliers:", y_semout[y_semout['Top'] == -1].shape[0])
    print("Número de colunas sem outliers:", df.shape[0])
    df.drop('id', axis=1, inplace=True)
    df.fillna(0, inplace=True)
    return df

df = remove_outliers(df)

Número de outliers: 3079
Número de colunas sem outliers: 18534


# Analise exploratória 

Para visualizar o comportamento de cada variável do dataset (preço, número de banheiros, números de quartos, área da casa e do seu terreno, etc) foram plotados gráficos para identificar a distribuição de cada uma delas dentro do dataset. Além disso, como a variável de interesse para prever é o preço de casas, foram plotados alguns gráficos para visualizar a interação entre a coluna de preços das casas e as demais colunas. 

## Distribuição de cada variável

Abaixo está os gráficos da distribuição de cada variável, exceto da coluna com os preços, ela será analisada posteriormente:

In [28]:
## ADD DATAVIZ

# Referências

[Predicao-de-precos-de-imoveis-atraves-de-aprendizagem-de-maquina](https://www.researchgate.net/profile/Joao_Malere/publication/335392421_Predicao_de_precos_de_imoveis_atraves_de_aprendizagem_de_maquina/links/5d62abce92851c619d76a46e/Predicao-de-precos-de-imoveis-atraves-de-aprendizagem-de-maquina.pdf)

[Avaliacao-de-Imoveis-Urbanos-com-Utilizacao-de-Redes-Neurais-Artificiais.pdf](http://ibape-nacional.com.br/biblioteca/wp-content/uploads/2012/12/Avaliacao-de-Imoveis-Urbanos-com-Utilizacao-de-Redes-Neurais-Artificiais.pdf)

[COMPARAÇÃO ENTRE AS TÉCNICAS DE REGRESSÃO](http://www.coordest.ufpr.br/wp-content/uploads/2018/12/TCC_DanielEricson.pdf)

[DETERMINANTS OF REAL HOUSE PRICE DYNAMICS](https://www.nber.org/papers/w9262.pdf)

[algoritmo-randomforest](https://didatica.tech/o-que-e-e-como-funciona-o-algoritmo-randomforest/)

[modelagem-baseada-em-tree-arvore](https://www.vooo.pro/insights/um-tutorial-completo-sobre-a-modelagem-baseada-em-tree-arvore-do-zero-em-r-python/#:~:text=As%20%C3%A1rvores%20de%20regress%C3%A3o%20s%C3%A3o,valor%20m%C3%A9dio%20das%20suas%20observa%C3%A7%C3%B5es.)

[hyperparameter-tuning](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

[linear-regression-decision-tree](https://medium.com/analytics-vidhya/linear-regression-decision-tree-and-ensemble-learning-applied-to-seoul-housing-prices-830d3493cfdb)

[Algoritmo Gradient Boosting](http://repositorio.ufjf.br:8080/jspui/bitstream/ufjf/3563/1/victorteixeirademelomayrink.pdf)